### Import Dependencies 

In [1]:
import cv2
import os
import mediapipe as mp 
import numpy as np 
import csv

### Set Root Directory for The Dataset Folder


In [2]:
#dataset path
root_dir = 'E:/data/graduation project/emotion detection using emotic/'


### Map Image Folders to Labels


In [3]:
# Initialize list to store the labels
labels = []

# Define a dictionary to map the folder names to labels
label_dict = {'Aversion' : 'Aversion', 'Annoyance' : 'Annoyance', 'Engagement': 'Engagement', 'Excitement': 'Excitement', 'Disconnection' : 'Disconnection', 'Pleasure':'Pleasure'}

### Create List of Column Headings for CSV File

In [4]:
# Create a list of column names for the class label and the different sets of coordinates, including x, y, z, 
# and v (visibility) for each set

coords_num = 501 # the number of landmarks detected by Mediapipe for the face (468) and the pose (33) 
landmarks = ['class']
for value in range(1, coords_num+1):
    landmarks += ['x{}'.format(value), 'y{}'.format(value),'z{}'.format(value), 'v{}'.format(value)]

### Create CSV File 

In [10]:
with open('emotic.csv', mode='w', newline='') as coords:
    csv_writer = csv.writer(coords, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Extract and Save Landmarks from FER13 Dataset into CSV File 

In [ ]:
# initializing the landmarks drawing utilites 
mp_drawing = mp.solutions.drawing_utils
# mediapipe landmark detection models
mp_holistic = mp.solutions.holistic

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Traverse the directory tree
    for root, dirs, files in os.walk(root_dir):
        print(root)
        for file in files:
            # Check if the file is an image
            if file.endswith('.jpg'):
                # Get the label from the parent directory
                label = os.path.basename(root)
                # Check if the label is in the dictionary
                if label in label_dict.values():
                    # Construct the full file path
                    file_path = os.path.join(root, file)
                    # Read the image using OpenCV
                    image = cv2.imread(file_path)
                    # Pass the image to medipipe for detection 
                    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    # Draw face laandmarks 
                    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                                  )
                    # Draw right hand laandmarks 
                    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                                  )
                    # Draw left hand laandmarks 
                    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                                  )
                    # Draw left hand laandmarks 
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                                  )

                    # Export Coordinates
                    try:
                        # Extract pose landmarks
                        pose = results.pose_landmarks.landmark
                        pose_row= list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]for landmark in pose]).flatten())

                        # Extract face landmarks
                        face = results.face_landmarks.landmark
                        face_row= list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]for landmark in face]).flatten())

                        # concatenate rows
                        row = pose_row + face_row

                        # Append class name
                        row.insert(0, label_dict[label])

                        # Export to CSV
                        with open('emotic.csv', mode='a', newline='') as coords:
                            csv_writer = csv.writer(coords, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            csv_writer.writerow(row)   

                    except:
                        pass
    